In [1]:
import pandas as pd
import numpy as np
import time
import streamlit as st
import functools

In [2]:
board_size = 19
win_length = 5
player = 1
score = {1: 0, -1: 0}
valeurs_possibles = [-1, 0, 1]

In [143]:
# Générer 100 tableaux 19x19
tableaux = [np.random.choice(valeurs_possibles, size=(19, 19)) for _ in range(100)]

# Affichage d'un exemple de tableau
print(tableaux[0])

[[ 0  0  0 -1  0  1  1 -1  0 -1 -1  1 -1 -1 -1 -1  0 -1  1]
 [ 1 -1 -1 -1 -1 -1  1  1  0  0 -1  0 -1 -1  1  0 -1  0  1]
 [-1  1 -1  1  1  1  1  1  0  1  0 -1  1 -1 -1  1 -1  1  1]
 [ 0  0  1  1  0 -1  0 -1  0  1  0 -1 -1  0 -1 -1  1  0 -1]
 [-1  1 -1  1  0  1 -1  0  0 -1  0  0 -1  1 -1  0  1  0 -1]
 [ 0  1 -1  1  1  0  1 -1  0  1 -1 -1 -1  1 -1 -1  0 -1  1]
 [ 0  1 -1 -1 -1  0  1  0  1  1  1  1  1  0  1  0 -1  1 -1]
 [-1  1 -1  0  1  1  0  0  0  0  1  0  1 -1  0  1  1 -1  0]
 [ 1  1  1  1  1  1  0  1  1 -1 -1 -1 -1 -1  0  1 -1  0  0]
 [ 0 -1 -1  1  0 -1 -1  1  1 -1  0 -1  1  1 -1 -1  0  1  0]
 [ 1 -1  0  1  0 -1  0  0  0 -1  1 -1  0 -1 -1  1  0 -1  0]
 [ 0 -1 -1 -1 -1  0 -1 -1 -1  1 -1 -1  0  1  0  0  1  1  0]
 [ 0  1  1  1 -1  0 -1  0 -1  0  0  0 -1  0  0  1  0  0  0]
 [ 1  1  1  0 -1  1  1 -1  0  1 -1  0  0  0 -1 -1 -1  0  0]
 [-1 -1  1 -1  0 -1  1  0 -1  0 -1  0  0  1 -1  0 -1  1  1]
 [-1  0  0  1  1  0 -1 -1  0 -1  1 -1  1  1 -1  1  1  1  1]
 [-1 -1  0 -1 -1  1  1 -1  1  0  0  1  0

In [153]:
%cd ..
from src.heuristic import heuristic
%load_ext autoreload
%autoreload 2

/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
tableaux = [np.random.choice(valeurs_possibles, size=(19, 19)) for _ in range(1000)]
start = time.time()
print("Start")
for i in range(1000):
    res = heuristic(board_size, win_length, tableaux[i], player, 2, 2, score)
end = time.time()
elapsed = end -start
print(f"End = {elapsed:.2}ms")

Start
End = 0.74ms


In [3]:
board_size = 8
tab  = [[0 for _ in range(board_size)] for _ in range(board_size)]
tab = np.array(tab)

tab[2][4] = -1
tab[3][3] = -1
# tab[4][4] = 1
tab[5][1] = -1
tab[5][3] = 1
tab[6][2] = 1
# tab[1][0] = -1
# tab[1][1] = 0
# tab[1][2] = 1
# tab[1][3] = 1
# tab[2][3] = 1
# tab[1][0] = 1
# tab[2][0] = 1
# tab[3][0] = 1
print(tab)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


In [30]:
row = 4
col = 4

tab2 = np.array(tab)
tab2[row][col] = player
res = heuristic(board_size, win_length, tab2, player, row, col, score)
print(f"RESULTAT {res}")

   Point 4 4
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0 -1  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
(2)(4) = -1
 -> (2)(4) = -1
 -> (3)(5) = 0
(open 2 k 1)
(3)(3) = -1
 -> (3)(3) = -1
(open 1 k 1)
(5)(1) = -1
 -> (5)(1) = -1
(open 1 k 1)
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0]
 [ 0  0  0 -1  0  1  0  0]
 [ 0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0]]
(3)(2) = -1
 -> (3)(2) = -1
 -> (4)(3) = -1
 -> (5)(4) = 0
 -> (6)(5) = -1
 -> (7)(6) = 0
(open 2 k 3)
-5.0
RESULTAT -5.0


In [17]:
%cd ..
from src.search_algo import greedy_best_first
%load_ext autoreload
%autoreload 2

2025-06-17 19:55:49.378 No runtime found, using MemoryCacheStorageManager


/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
greedy_best_first(tab, board_size, win_length, heuristic, player, depth_limit=10, score=None, debug=True, game_rules=["Capture", "Double Three"])

   Point 4 0
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0  0]
 [ 1  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
-0.05000000000000002
   Point 3 4
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
-0.05000000000000002
   Point 4 3
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
-0.040000000000000015
   Point 3 1
[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  1  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
-0.0500000000000

(4, 4)

In [162]:
board_size = 8
tab  = [[0 for _ in range(board_size)] for _ in range(board_size)]
tab = np.array(tab)

tab[2][2] = 1
tab[3][3] = -1
tab[4][4] = -1
# tab[5][1] = -1
print(tab)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


In [ ]:
row = 5
col = 5

tab2 = np.array(tab)
tab2[row][col] = player
res = heuristic(board_size, win_length, tab2, player, row, col, score)
print(f"RESULTAT {res}")

/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
RESULTAT 0.010750000000000001


In [108]:
win_len = 4
reward_closed = [0] * (win_len + 1)
reward_open1 = [0] + [0] + [10**i for i in range(-2, 2 * win_len - 4, 2)]
reward_open2 = [0] + [0] + [10**i for i in range(-1, 2 * win_len - 5, 2)]
reward_closed[win_len] = 10**(win_len)
reward_open1[win_len] = 10**(win_len)
reward = [reward_closed, reward_open1, reward_open2]
reward_block = [[val * 1.5 for val in lst] for lst in reward]
reward_np = [np.array(lst) * 1.5 for lst in reward]
reward = [reward, reward_block]
print(reward_block)
# print(reward_block)
# print(reward_np)
# print(reward[0][0][1])

[[0.0, 0.0, 0.0, 0.0, 15000.0], [0.0, 0.0, 0.015, 1.5, 15000.0], [0.0, 0.0, 0.15000000000000002, 15.0]]


In [ ]:
def get_oponent(val, a, b, win_len):
    if val == 10**(win_len):
        return -val
    return -val * a + b

def get_reward(win_len):
    reward_closed = [0] * (win_len + 1)
    reward_open1 = [0] + [10**i for i in range(-4, 2 * win_len - 4, 2)]
    reward_open2 = [0] + [10**i for i in range(-3, 2 * win_len - 3, 2)]
    reward_closed[win_len] = 10**(2 * win_len - 5)
    reward_open1[win_len] = 10**(2 * win_len - 5)
    reward = [reward_closed, reward_open1, reward_open2]
    reward_open1[win_len] = 10**(2 * win_len - 5)
    reward_open2[win_len] = 10**(2 * win_len - 5)
    reward = [reward_closed, reward_open1, reward_open2]
    return reward

In [140]:
i = 3
while i < 10:
    print(get_reward(i))
    i += 1

[[0, 0, 0, 10], [0, 0.0001, 0.01, 10], [0, 0.001, 0.1, 10]]
[[0, 0, 0, 0, 1000], [0, 0.0001, 0.01, 1, 1000], [0, 0.001, 0.1, 10, 1000]]
[[0, 0, 0, 0, 0, 100000], [0, 0.0001, 0.01, 1, 100, 100000], [0, 0.001, 0.1, 10, 1000, 100000]]
[[0, 0, 0, 0, 0, 0, 10000000], [0, 0.0001, 0.01, 1, 100, 10000, 10000000], [0, 0.001, 0.1, 10, 1000, 100000, 10000000]]
[[0, 0, 0, 0, 0, 0, 0, 1000000000], [0, 0.0001, 0.01, 1, 100, 10000, 1000000, 1000000000], [0, 0.001, 0.1, 10, 1000, 100000, 10000000, 1000000000]]
[[0, 0, 0, 0, 0, 0, 0, 0, 100000000000], [0, 0.0001, 0.01, 1, 100, 10000, 1000000, 100000000, 100000000000], [0, 0.001, 0.1, 10, 1000, 100000, 10000000, 1000000000, 100000000000]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 10000000000000], [0, 0.0001, 0.01, 1, 100, 10000, 1000000, 100000000, 10000000000, 10000000000000], [0, 0.001, 0.1, 10, 1000, 100000, 10000000, 1000000000, 100000000000, 10000000000000]]


In [140]:
board_size = 6
tab  = [[0 for _ in range(board_size)] for _ in range(board_size)]
tab = np.array(tab)

tab[0][0] = -1
tab[1][1] = 1
tab[2][2] = 1
tab[5][5] = -1
# tab[4][4] = -1
# tab[3][0] = 1
# tab[5][0] = -1
print(tab)

[[-1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0 -1]]


In [143]:
row = 3
col = 3
win_length = 5
player = 1
score = {1: 0, -1: 0}

%cd ..
from src.heuristic import heuristic
%load_ext autoreload
%autoreload 2

tab[row][col] = player
print(tab)
res = heuristic(board_size, win_length, tab, player, row, col, score)
print(f"RESULTAT {res}")

/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[-1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0 -1]]
RESULTAT -0.9917499999999999
